## Q1 How many transactions in the dataset?

## Q2 What is the top-3 most expensive item in the database? 
* show the price of item.

## Q3 Show number of customers were born in each month.
* there are 12 months

## Q4 Show top-3 popular items that make most income.
* show the total income of each item.

## Q5 A manager want to crate a birth-month promotion in January 2017.
* How many customers were born in January and also made transcations in January 2017?

## Q6 In January, what were the top-3 favorite items that customer bought with "Bread"?
* show number of times

## Q7 Who are the top-3 customers that pay most money for "Tea"? 
* show total payment of each customer.

## Q8 What are the top-5 items that "Harry Brown" pay most money? 
* show total payment of each item.

## Q9 Who are top-5 customer paid most money in 2016? 
* show total payment of each customer

## Q10 Between January and March 2017, Who are top-3 customers that paid most for Coffee?
* show total payment

In [34]:
# import sqlite3
# import pandas as pd
# con = sqlite3.connect('db.sqlite3')

In [36]:
import psycopg2 as pg
import pandas as pd
 
# get connected to the database
con = pg.connect("host='bem.ei.team' dbname='myproject' user='myprojectuser' password='password' port='5432'")
dataframe = pd.read_sql_query("SELECT * FROM myapp_item", con)
dataframe.head()

,id,name,price
0,1,Bread,380.0
1,2,Scandinavian,320.0
2,3,Hot chocolate,390.0
3,4,Jam,210.0
4,5,Cookies,210.0


In [38]:
df_transaction = pd.read_sql(
    sql="SELECT * FROM myapp_transaction",
    con=con,
    parse_dates={'datetime': {'format': '%Y-%m-%d %H:%M:%S'}},
    index_col ='id'
)
df_transaction.head()

,datetime,customer_id
id,,
1,2016-10-30 09:58:11+00:00,27
2,2016-10-30 10:05:34+00:00,92
3,2016-10-30 10:07:57+00:00,73
4,2016-10-30 10:08:41+00:00,94
5,2016-10-30 10:13:03+00:00,100


In [39]:
df_customer = pd.read_sql(
    sql="SELECT * FROM myapp_customer",
    con=con,
    parse_dates={'dob': {'format': '%Y-%m-%d'}},
    index_col ='id'
)
df_customer.head()

,firstname,lastname,dob
id,,,
1,Jack,Jones,1969-10-03
2,Oscar,Taylor,1938-05-20
3,Bob,Davies,1940-06-03
4,Charlie,Smith,1951-06-16
5,Cath,Brown,1926-02-17


In [40]:
df_record = pd.read_sql(
    sql="SELECT * FROM myapp_record",
    con=con,
    index_col ='id'
)
df_record.head()

,item_id,transaction_id
id,,
1,1,1
2,2,2
3,2,2
4,3,3
5,4,3


In [41]:
df_item = pd.read_sql(
    sql="SELECT * FROM myapp_item",
    con=con,
    index_col ='id'
)
df_item.head()

,name,price
id,,
1,Bread,380.0
2,Scandinavian,320.0
3,Hot chocolate,390.0
4,Jam,210.0
5,Cookies,210.0


In [42]:
#Q1 How many transactions in the dataset?
df_transaction.count()

datetime       4704
customer_id    4704
dtype: int64

In [43]:
#Q2 What is the top-3 most expensive item in the database?
#show the price of item.
df_item.sort_values(by=['price'],ascending=False).head(3)

,name,price
id,,
48,Bacon,400.0
39,Fairy Doors,400.0
73,Salad,390.0


In [44]:
#Q3 Show number of customers were born in each month.
#there are 12 months
df_customer['month']=df_customer.dob.dt.month
df_customer.groupby(by=['month']).agg({'month':['count']})

,month
,count
month,
1,13
2,9
3,8
4,6
5,12
6,7
7,9
8,13


In [45]:
#Q4 Show top-3 popular items that make most income.¶
#show the total income of each item.
df_record.merge(right=df_item,left_on='item_id',right_index=True)\
    .groupby('name')\
    .agg({'price':['sum']})\
    .sort_values(('price','sum'),ascending=False)\
    .head(3)

,price
,sum
name,
Coffee,733880.0
Bread,624340.0
Tea,248640.0


In [46]:
#Q5 A manager want to crate a birth-month promotion in January 2017.
#How many customers were born in January and also made transcations in January 2017?
df1=df_customer[df_customer.month==1]
df_transaction['month_transaction']=df_transaction.datetime.dt.month
df2=df_transaction[df_transaction.month_transaction==1]

df1.merge(
    right=df2,
    left_index=True,
    right_on='customer_id'
).groupby(by=['customer_id'])\
.agg({'customer_id':['count']})\
.count()

customer_id  count    13
dtype: int64

In [47]:
# Q6 In January, what were the top-3 favorite items that customer bought with "Bread"?
#* show number of times


df_all=df_record.merge(right=df_item,left_on='item_id',right_index=True)\
    .merge(right=df_transaction,left_on='transaction_id',right_index=True)

df_all[
        (df_all.datetime.dt.month==1) &\
        (df_all.name=='Bread')
    ]\
    .groupby(['item_id'])\
    .agg({'transaction_id':['count']})\
    .sort_values(('transaction_id','count'),ascending=False)\
.head()

#df_all.head()
# \
#     .groupby(['item_id'])\
#     .agg({'price':['count']})\
#     .sort_values(('price','sum'),ascending=False)\
#     .head(3)

,transaction_id
,count
item_id,
1,246


In [16]:
df_record.merge(right=df_item,left_on='item_id',right_index=True).head()

,item_id,transaction_id,name,price
id,,,,
1,1,1,Banana,50
2,1,1,Banana,50
7,1,3,Banana,50
3,2,1,Apple,60
5,2,2,Apple,60


In [48]:
# Q7 Who are the top-3 customers that pay most money for "Tea"? 
#* show total payment of each customer.
df_all=df_record.merge(right=df_item,left_on='item_id',right_index=True)\
    .merge(right=df_transaction,left_on='transaction_id',right_index=True)\
    .merge(right=df_customer,left_on='customer_id',right_index=True)\

df_all[df_all.name=='Tea']\
    .groupby(['customer_id','firstname','lastname'])\
    .agg({'price':['sum']})\
    .sort_values(('price','sum'),ascending=False)\
    .head(3)

,,,price
,,,sum
customer_id,firstname,lastname,
119,William,Smith,4440.0
108,Bob,Taylor,4070.0
8,Jack,Evans,4070.0


In [49]:
#Q8 What are the top-5 items that "Harry Brown" pay most money?
#show total payment of each item.

df_all=df_record.merge(right=df_item,left_on='item_id',right_index=True)\
    .merge(right=df_transaction,left_on='transaction_id',right_index=True)\
    .merge(right=df_customer,left_on='customer_id',right_index=True)\

df_all[(df_all.firstname=='Harry') & (df_all.lastname=='Brown')]\
    .groupby(['item_id','name'])\
    .agg({'price':['sum']})\
    .sort_values(('price','sum'),ascending=False)\
    .head(5)

,,price
,,sum
item_id,name,
7,Coffee,7280.0
1,Bread,4940.0
9,Medialuna,2220.0
8,Pastry,1610.0
10,Tea,1480.0


In [50]:
#Q9 Who are top-5 customer paid most money in 2016?
#show total payment of each customer
df_all=df_record.merge(right=df_item,left_on='item_id',right_index=True)\
    .merge(right=df_transaction,left_on='transaction_id',right_index=True)\
    .merge(right=df_customer,left_on='customer_id',right_index=True)\

df_all[df_all.datetime.dt.year==2016]\
    .groupby(['customer_id','firstname','lastname'])\
    .agg({'price':['sum']})\
    .sort_values(('price','sum'),ascending=False)\
    .head(5)

,,,price
,,,sum
customer_id,firstname,lastname,
107,James,Taylor,36340.0
116,Claire,Evans,33190.0
17,James,Smith,33040.0
32,Alex,Brown,31380.0
93,Charlie,Johnson,31030.0


In [51]:
#Q10 Between January and March 2017, Who are top-3 customers that paid most for Coffee?¶
#show total payment

df_all=df_record.merge(right=df_item,left_on='item_id',right_index=True)\
    .merge(right=df_transaction,left_on='transaction_id',right_index=True)\
    .merge(right=df_customer,left_on='customer_id',right_index=True)\

df_all[
        (df_all.datetime.dt.year==2017) &\
        (df_all.datetime.dt.month>=1) &\
        (df_all.datetime.dt.month<=3) &\
        (df_all.name=='Coffee')
    ]\
    .groupby(['customer_id','firstname','lastname'])\
    .agg({'price':['sum']})\
    .sort_values(('price','sum'),ascending=False)\
    .head(3)

,,,price
,,,sum
customer_id,firstname,lastname,
68,Harry,Taylor,2520.0
4,Charlie,Smith,2520.0
81,Thomas,Johnson,2520.0
